## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [4]:
data = list(range(1,1001))
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [5]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [6]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [7]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [8]:
# count
rdd.count()

1000

In [9]:
# first
rdd.first()

1

In [10]:
# take
rdd.take(10)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [11]:
# top
rdd.top(10)

[1000, 999, 998, 997, 996, 995, 994, 993, 992, 991]

In [12]:
# collect
## Note: When you are dealing with big data, this could make your computer crash! It's best to avoid using the collect() method
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [13]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([3.70690806e-01, 1.23905372e+00, 7.65850452e-01, 2.77685743e+00,
       3.11850383e+00, 1.84372535e-02, 1.86091533e+00, 7.41485515e-01,
       1.87650622e+00, 3.23912437e+00, 1.01997966e+01, 3.69891806e+00,
       1.21943165e+01, 2.69307291e+00, 4.90812025e+00, 6.03818207e+00,
       6.19495847e+00, 1.52872043e+00, 1.19403142e+01, 2.56145192e+00,
       1.02339192e+01, 1.34909094e+01, 2.17208372e+01, 1.00473660e+01,
       5.31884797e+00, 2.77425942e+00, 2.07529222e+01, 2.58111553e+01,
       2.27626099e+01, 6.74448366e+00, 9.71263037e+00, 1.51795744e+01,
       3.11064903e+01, 1.27791373e+01, 3.09974530e+01, 3.24222273e+01,
       6.34757776e+00, 3.05196761e+00, 1.78032866e+01, 2.11445317e+01,
       3.71852446e+01, 3.85244974e+01, 3.16587512e+01, 3.24956926e+01,
       2.06390140e+01, 2.66822612e+01, 1.30114845e+01, 4.20693153e+01,
       4.22700502e-01, 1.75198645e+01, 1.49136799e+01, 3.93945188e+01,
       2.34236363e+01, 5.37095694e+01, 5.35259694e+01, 2.28889817e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [14]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.take(4)

[0.3706908060224924, 1.2390537233168852, 0.7658504519735302, 2.776857430442798]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [15]:
def sales_tax(num):
    return num * 0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [16]:
# perform action to retrieve rdd values
revenue_minus_tax.take(10)

[0.341035541540693,
 1.1399294254515344,
 0.7045824158156478,
 2.554708836007374,
 2.8690235273981193,
 0.016962273255195708,
 1.7120421000946116,
 0.6821666740623639,
 1.7263857205983912,
 2.9799944217975036]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [17]:
discounted = revenue_minus_tax.map(lambda x : x*0.9)

In [18]:
discounted.take(10)

[0.3069319873866237,
 1.025936482906381,
 0.6341241742340831,
 2.299237952406637,
 2.5821211746583073,
 0.015266045929676137,
 1.5408378900851505,
 0.6139500066561275,
 1.5537471485385521,
 2.681994979617753]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [19]:
price_items.map(sales_tax).map(lambda x : x*0.9).top(15)

[819.1244814518534,
 776.1585593097204,
 765.0298778923354,
 762.3339645595623,
 754.1050685096099,
 747.0670058517768,
 746.2871288187974,
 744.5939435145664,
 730.2224021608461,
 720.3436784289435,
 708.9300446880595,
 708.7232226031688,
 697.8275757125386,
 697.5081725914326,
 696.2406028786088]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [20]:
discounted.toDebugString()

b'(10) PythonRDD[10] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[5] at readRDDFromFile at PythonRDD.scala:262 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [21]:
mapped = price_items.map(lambda x: (x, x*0.92 *0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.3706908060224924, 0.3069319873866237), (1.2390537233168852, 1.025936482906381), (0.7658504519735302, 0.6341241742340831), (2.776857430442798, 2.299237952406637), (3.1185038341283904, 2.5821211746583073), (0.018437253538256204, 0.015266045929676137), (1.860915326189795, 1.5408378900851505), (0.741485515285178, 0.6139500066561275), (1.8765062180417296, 1.5537471485385521), (3.2391243715190257, 2.681994979617753)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [22]:
flat_mapped = price_items.flatMap(lambda x : (x, x*0.92*0.9 ))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.3706908060224924, 0.3069319873866237, 1.2390537233168852, 1.025936482906381, 0.7658504519735302, 0.6341241742340831, 2.776857430442798, 2.299237952406637, 3.1185038341283904, 2.5821211746583073]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [23]:
# use the filter function
selected_items = discounted.filter(lambda x: x>300)
selected_items.count()

# calculate total remaining in inventory 


260

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [24]:
selected_items.reduce(lambda x,y: x + y)

122200.61395487012

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [25]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(46, 325.94057128826216),
 (31, 329.19856907254996),
 (16, 303.1030056785406),
 (21, 301.2483215365439),
 (47, 319.84369035133784),
 (46, 349.365878792807),
 (47, 355.72822426700003)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [26]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x, y: x + y)
total_spent.take(10)

[(50, 3448.3715033823464),
 (30, 2301.474621219649),
 (10, 3538.036651711441),
 (20, 1652.7104387994082),
 (40, 1492.5563667204083),
 (41, 2298.915280177093),
 (31, 4420.059031358298),
 (1, 2078.4553678028233),
 (11, 2746.430501599841),
 (21, 2956.0772308842966)]

In [27]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1],ascending = False).collect()

[(9, 4944.764907947965),
 (12, 4639.889549852505),
 (31, 4420.059031358298),
 (37, 4004.6099302155862),
 (32, 3779.250158030902),
 (24, 3761.8577287292187),
 (10, 3538.036651711441),
 (50, 3448.3715033823464),
 (4, 3237.2614133305624),
 (44, 3179.3259348276965),
 (36, 3090.4219394044176),
 (48, 3051.501044524571),
 (21, 2956.0772308842966),
 (33, 2930.9926548008607),
 (14, 2838.9017358941364),
 (5, 2822.207217591256),
 (11, 2746.430501599841),
 (16, 2728.22596702544),
 (29, 2705.1512255879925),
 (15, 2645.873359336159),
 (46, 2557.395181532216),
 (38, 2513.490916322494),
 (26, 2367.4898837012747),
 (34, 2362.450952774194),
 (28, 2352.6040763229),
 (30, 2301.474621219649),
 (41, 2298.915280177093),
 (39, 2287.6301318719597),
 (43, 2217.315524200727),
 (7, 2198.614367172534),
 (25, 2177.3475189046894),
 (1, 2078.4553678028233),
 (35, 2076.12787783719),
 (8, 2062.4249625596076),
 (19, 1998.406876115743),
 (23, 1878.4021090697547),
 (49, 1744.2706510263097),
 (2, 1720.4747537495427),
 (45,

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [28]:
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda kv:kv[1],reverse=True)

[(10, 11),
 (45, 10),
 (25, 10),
 (8, 9),
 (21, 9),
 (42, 9),
 (41, 9),
 (5, 9),
 (22, 7),
 (37, 7),
 (11, 7),
 (19, 7),
 (50, 7),
 (39, 7),
 (33, 7),
 (14, 6),
 (6, 6),
 (17, 6),
 (16, 6),
 (29, 5),
 (23, 5),
 (47, 5),
 (28, 5),
 (32, 5),
 (49, 5),
 (46, 4),
 (48, 4),
 (2, 4),
 (43, 4),
 (44, 4),
 (40, 4),
 (34, 4),
 (35, 4),
 (18, 4),
 (31, 4),
 (15, 4),
 (4, 3),
 (1, 3),
 (7, 3),
 (12, 3),
 (20, 3),
 (27, 3),
 (30, 3),
 (26, 3),
 (24, 3),
 (13, 3),
 (36, 2),
 (38, 2),
 (3, 2),
 (9, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
